<a href="https://colab.research.google.com/github/CaosFq/Entrega_Final/blob/main/Entrega_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Etapa 1: Recopilaciòn y Preparaciòn de Datos**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving clientes.csv to clientes (2).csv
Saving marketing.csv to marketing (2).csv
Saving ventas.csv to ventas (2).csv


In [ ]:
import pandas as pd
import numpy as np

ventas = pd.read_csv("ventas.csv")
clientes = pd.read_csv("clientes.csv")
marketing = pd.read_csv("marketing.csv")


Anàlisis Exploratorio Inicial

In [ ]:
def eda(df, nombre):
    print(f"=== {nombre} ===")
    print("Shape:", df.shape)
    print("Columnas:", list(df.columns))
    print("\nTipos de datos:")
    print(df.dtypes)
    print("\nNulos por columna:")
    print(df.isna().sum())
    print("\nPrimeras filas:")
    display(df.head())
    print("\nResumen numérico:")
    display(df.describe(include='number'))
    print("-"*80)





In [ ]:
eda(ventas, "VENTAS (inicial)")

In [ ]:
eda(clientes, "CLIENTES (inicial)")

In [ ]:
eda(marketing, "MARKETING (inicial)")

Calidad de Datos: Identificar valores nulos y duplicados en los valores iniciales

In [ ]:
def calidad(df, nombre, clave=None):
    # Intentar importar display (si estás en Colab o Jupyter)
    try:
        from IPython.display import display
        usa_display = True
    except ImportError:
        usa_display = False

    print(f"\n=== 🔍 CALIDAD DE DATOS: {nombre.upper()} ===")

    # --- Nulos por columna ---
    nulos = df.isna().sum().to_frame("Nulos")
    print("--- 1. Nulos por Columna ---")
    if usa_display:
        display(nulos)
    else:
        print(nulos)

    # --- Filas duplicadas completas ---
    print("\n--- 2. Duplicados ---")
    dup_rows = df.duplicated(keep=False).sum()
    print(f"🧩 Filas duplicadas (exactas): {dup_rows}")

    # --- Duplicados por clave (si aplica) ---
    if clave and clave in df.columns:
        dup_key = df[clave].duplicated(keep=False).sum()
        print(f"🔑 Duplicados por clave '{clave}': {dup_key}")

        if dup_key > 0:
            top_dup = (
                df[df[clave].duplicated(keep=False)][clave]
                .value_counts()
                .head(10)
            )
            print("\n📋 Top valores duplicados más frecuentes:")
            if usa_display:
                display(top_dup)
            else:
                print(top_dup)
        else:
            print(f"✅ No se encontraron duplicados en la clave '{clave}'.")
    elif clave:
        print(f"⚠️ La clave '{clave}' no existe en el DataFrame.")
    else:
        print("ℹ️ No se indicó una clave para analizar duplicados.")
    print("-" * 50)

In [ ]:

calidad(ventas, "VENTAS", clave="id_venta")


In [ ]:
calidad(clientes, "CLIENTES", clave="id_cliente")

In [ ]:
calidad(marketing, "MARKETING", clave="id_cliente")

**Etapa 2: Procesamiento de Datos**

Eliminaciòn de Duplicados:

In [ ]:
# Diccionario con los DataFrames originales
dfs = {'ventas': ventas, 'clientes': clientes, 'marketing': marketing}
# Crear copias limpias y eliminar duplicados
dfs_clean = {name: df.drop_duplicates().copy() for name, df in dfs.items()}

Funciòn normalizar_texto(df)

In [ ]:

def normalizar_texto(df):
    for col in df.select_dtypes(include="object").columns:
        df[col] = (
            df[col]
            .astype(str)                              # Convertir todo a texto
            .str.strip()                               # Quitar espacios al inicio y final
            .str.replace(r"[\u200b\t\r\n]", "", regex=True)  # Eliminar caracteres invisibles
            .str.replace(r" +", " ", regex=True)      # Reducir espacios múltiples a uno
            .str.title()                               # Convertir a formato título
        )
    return df


Normalización de Fechas

In [ ]:
# 1.DataFrames limpios
dfs = {'ventas': ventas, 'clientes': clientes, 'marketing': marketing}
dfs_clean = {name: df.drop_duplicates().copy() for name, df in dfs.items()}

# 2. Sobrescribir los originales con los limpios
# Esta es la forma de "mantener el nombre ventas":
ventas = dfs_clean['ventas']
clientes = dfs_clean['clientes']
marketing = dfs_clean['marketing']

# 3. Ajustar tu bucle de fechas para usar los nombres de variables sin el "_clean"
for df in [ventas, clientes, marketing]: # <--- ¡Cambiamos aquí!
    for col in df.columns:
        if "fecha" in col.lower():
            df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)

In [ ]:
print(ventas.dtypes)
print(clientes.dtypes)
print(marketing.dtypes)


Limpieza de Precio y Cantidad

In [ ]:
print("--- 📋 Nombres EXACTOS de Columnas en 'ventas' ---")
print(ventas.columns.tolist())

# Imprime también los tipos de datos para ver si ya están como object o si son numéricos
print("\n--- Tipos de Datos Actuales ---")
print(ventas.dtypes)

In [ ]:
print("--- 📋 Nombres EXACTOS de Columnas en 'ventas' ---")
print(ventas.columns.tolist())

In [ ]:
# ⚠️ Nombres Reales ENCONTRADOS
NOMBRE_REAL_PRECIO = 'precio'
NOMBRE_REAL_CANTIDAD = 'cantidad'

print(f"\n--- 🔄 Forzando Conversión de '{NOMBRE_REAL_PRECIO}' y '{NOMBRE_REAL_CANTIDAD}' ---")

# --- 1. Limpieza y Conversión para PRECIO ---
if NOMBRE_REAL_PRECIO in ventas.columns:
    # 1. Limpieza de texto agresiva
    ventas[NOMBRE_REAL_PRECIO] = (
        ventas[NOMBRE_REAL_PRECIO]
        .astype(str)
        # Quitar símbolos comunes ($, €, %, etc.) y letras/espacios no numéricos
        .str.replace('[^0-9\.\,-]', '', regex=True)
        .str.replace(',', '.', regex=False) # Si usan coma como separador decimal, cambiar a punto
        .str.strip()
    )

    # 2. Conversión a numérico (float)
    ventas[NOMBRE_REAL_PRECIO] = pd.to_numeric(ventas[NOMBRE_REAL_PRECIO], errors='coerce')
    ventas[NOMBRE_REAL_PRECIO].fillna(0, inplace=True)
    print(f"✅ Columna '{NOMBRE_REAL_PRECIO}' limpiada y convertida a numérico.")

# --- 2. Limpieza y Conversión para CANTIDAD ---
if NOMBRE_REAL_CANTIDAD in ventas.columns:
    # Conversión directa a numérico (asumiendo que tiene menos ruido)
    ventas[NOMBRE_REAL_CANTIDAD] = pd.to_numeric(ventas[NOMBRE_REAL_CANTIDAD], errors='coerce')
    ventas[NOMBRE_REAL_CANTIDAD].fillna(1, inplace=True) # Rellenar nulos con 1 (o 0)
    print(f"✅ Columna '{NOMBRE_REAL_CANTIDAD}' convertida a numérico.")

# 3. VERIFICACIÓN DE TIPO (Esto DEBE ser numérico)
print("\n--- ✅ Tipos de Datos después de la Conversión ---")
print(ventas[[NOMBRE_REAL_PRECIO, NOMBRE_REAL_CANTIDAD]].dtypes)

# 4. Cálculo final del Ingreso Total
ventas['Ingreso_Total'] = ventas[NOMBRE_REAL_PRECIO] * ventas[NOMBRE_REAL_CANTIDAD]

print("\n--- 💰 Ingreso Total Calculado con Éxito ---")
print(ventas[[NOMBRE_REAL_PRECIO, NOMBRE_REAL_CANTIDAD, 'Ingreso_Total']].head())

Integración

In [ ]:
print(ventas.columns.tolist())

In [ ]:
print("--- 📋 Nombres EXACTOS de Columnas para Agregación ---")
# Asumo que tu DataFrame se llama 'ventas' o 'df_integrado'
# Si estás usando 'df' y te da error, cambia el nombre aquí
print(ventas.columns.tolist())

In [ ]:

NOMBRE_REAL_CATEGORIA = 'categoria'

# Realizar la agregación con el nombre y el DataFrame correctos
resumen_por_categoria = (
    ventas.groupby(NOMBRE_REAL_CATEGORIA, as_index=False)['Ingreso_Total']
    .sum()
    .rename(columns={'Ingreso_Total': 'Ingreso_Total_Categoria'})
)

# Ordenar y mostrar resultados
resumen_por_categoria.sort_values(by='Ingreso_Total_Categoria', ascending=False, inplace=True)

print("\n--- ✅ Resumen de Ingresos por Categoría ---")
print(resumen_por_categoria.head(10))

In [ ]:
print("--- 📋 Nombres EXACTOS de Columnas en 'marketing' ---")
print(marketing.columns.tolist())

In [ ]:
print("--- 📋 Columnas de VENTAS ---")
print(ventas.columns.tolist())

print("\n--- 📋 Columnas de MARKETING ---")
print(marketing.columns.tolist())

Integración de Datos (por Producto)

In [ ]:
# ⚠️ Clave de Enlace Identificada
columna_clave_real = 'producto'

# 1. Aseguramos el tipo string para evitar errores de merge por tipos (aunque 'producto' ya es string)
ventas[columna_clave_real] = ventas[columna_clave_real].astype(str)
marketing[columna_clave_real] = marketing[columna_clave_real].astype(str)

# 2. Realizar la integración (Merge)
# Usamos how='left' para mantener todas las ventas y adjuntar información de marketing
df_integrado = pd.merge(
    left=ventas,
    right=marketing,
    on=columna_clave_real,
    how='left',
    suffixes=('_ventas', '_marketing')
)

print(f"\n--- 🤝 Integración Exitosa por '{columna_clave_real}' ---")
print(f"Forma inicial (Ventas): {ventas.shape}")
print(f"Forma final (Integrado): {df_integrado.shape}")
print("\nPrimeras 5 filas del DataFrame Integrado:")
print(df_integrado.head())

# ⚠️ Nota sobre esta integración: Si un producto aparece en múltiples campañas,
# la venta se duplicará por el número de campañas. Esto es normal y lo manejaremos en el análisis.

Etapa 3: Anàlisis de Datos

Estadistica Descriptiva

In [ ]:
## 📈 Estadística Descriptiva de Ingreso Total
print("\n=== 📊 ETAPA 3: ESTADÍSTICA DESCRIPTIVA ===")

# Calcular medidas de tendencia central y dispersión
analisis_ingreso = df_integrado['Ingreso_Total'].describe().to_frame("Ingreso Total (USD)")
analisis_ingreso.loc['mediana'] = df_integrado['Ingreso_Total'].median()
moda = df_integrado['Ingreso_Total'].mode()
analisis_ingreso.loc['moda'] = moda.iloc[0] if not moda.empty else 'N/A'

print(analisis_ingreso.applymap('{:,.2f}'.format))

Correlación entre Variables

In [ ]:
columnas_numericas_analisis = ['precio', 'cantidad', 'costo', 'Ingreso_Total']

# Filtrar solo las columnas
columnas_existentes = [col for col in columnas_numericas_analisis if col in df_integrado.columns]

# Calcular la matriz de correlación de Pearson
matriz_correlacion = df_integrado[columnas_existentes].corr()

print("\n=== 🔗 MATRIZ DE CORRELACIÓN DE PEARSON ===")
print(matriz_correlacion.round(2))

Interpretación

El valor de correlación (r) varía de -1 a 1:
Valor de r	Interpretación
+1	Correlación positiva perfecta (si uno sube, el otro sube).
0	No hay relación lineal.
-1	Correlación negativa perfecta (si uno sube, el otro baja).

Hallazgo Clave: la correlación entre costo (marketing) y Ingreso_Total.

    Si r≈1, la inversión en marketing está directamente relacionada con los ingresos.

    Si r≈0, la inversión no está teniendo un efecto lineal directo en el ingreso total por venta individual.

Análisis de Series de Tiempo

Preparación de la Serie de Tiempo

In [ ]:

COLUMNA_FECHA = 'fecha_venta'


df_integrado[COLUMNA_FECHA] = pd.to_datetime(df_integrado[COLUMNA_FECHA], errors='coerce')


df_series = df_integrado.set_index(COLUMNA_FECHA).copy()

ventas_mensuales = df_series['Ingreso_Total'].resample('MS').sum().to_frame('Ingreso_Total_Mensual')

print("\n=== 📅 TENDENCIA DE VENTAS MENSUALES (Top 5) ===")
print(ventas_mensuales.sort_index(ascending=False).head())

Análisis y Hallazgos (Mensual y Anual)

In [ ]:
## 📉 Análisis de Tendencias

# Tendencia Anual (Resample a 'AS' = Year Start)
ventas_anuales = df_series['Ingreso_Total'].resample('AS').sum().to_frame('Ingreso_Total_Anual')

print("\n--- Análisis Anual ---")
print(ventas_anuales)

# Hallazgos de Crecimiento
if ventas_anuales.shape[0] > 1:
    crecimiento = (ventas_anuales.iloc[-1] - ventas_anuales.iloc[-2]) / ventas_anuales.iloc[-2] * 100
    print(f"\n✅ Hallazgo de Crecimiento: Variación interanual (últimos dos años): {crecimiento.iloc[0]:.2f}%")
else:
    print("ℹ️ No hay suficientes datos para calcular el crecimiento interanual.")

# Identificación de Estacionalidad (Meses de mayor Venta)
ventas_por_mes = (
    df_series.groupby(df_series.index.month_name())['Ingreso_Total']
    .sum()
    .sort_values(ascending=False)
)

print("\n--- Estacionalidad (Ingreso por Nombre de Mes) ---")
print(ventas_por_mes.head(4))

print(f"\n✅ Hallazgo de Estacionalidad: El mes con mayor ingreso es **{ventas_por_mes.index[0]}**, lo que sugiere un pico estacional.")

Etapa 4: Visualización de Datos y Presentación de **Resultados**

Visualización de Series de Tiempo

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns



plt.figure(figsize=(14, 6))
sns.lineplot(data=ventas_mensuales, x=ventas_mensuales.index, y='Ingreso_Total_Mensual', marker='o')

# Añadir etiquetas y título
plt.title('Tendencia de Ingreso Total Mensual', fontsize=16)
plt.xlabel('Fecha')
plt.ylabel('Ingreso Total (USD)')
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Distribución de Ingresos por **Categoría**

In [ ]:

# Ordenar por ingreso de forma descendente para el gráfico
resumen_por_categoria.sort_values(by='Ingreso_Total_Categoria', ascending=False, inplace=True)

plt.figure(figsize=(10, 7))
sns.barplot(
    x='Ingreso_Total_Categoria',
    y='categoria',
    data=resumen_por_categoria,
    palette='viridis'
)

# Añadir etiquetas y título
plt.title('Contribución de Ingresos por Categoría de Producto', fontsize=16)
plt.xlabel('Ingreso Total (USD)')
plt.ylabel('Categoría')
plt.tight_layout()
plt.show()

Análisis de **Correlación**

In [ ]:


plt.figure(figsize=(8, 6))
sns.heatmap(
    matriz_correlacion,
    annot=True,     # Muestra los valores de correlación
    cmap='coolwarm', # Esquema de color
    fmt=".2f",      # Formato de dos decimales
    linewidths=.5   # Líneas entre celdas
)

plt.title('Mapa de Calor de la Correlación entre Variables', fontsize=16)
plt.tight_layout()
plt.show()

Etapa 5: Conclusiones y **Recomendaciones**

Mi Diagnóstico: ¿Qué Descubrí en los Datos?

Encontré tres mensajes clave que nos dan las ventas:

    El Volumen es el Rey: Cuando analicé la relación entre la cantidad de productos vendidos y el ingreso total, encontré una conexión extremadamente fuerte (0.77). Esto significa que si logramos que cada cliente ponga más cosas en su carrito, nuestro ingreso crecerá de forma inmediata y directa.

    La Venta es "Pequeña": Descubrí que el promedio de una venta es de $489.03, pero la mayoría de la gente gasta menos de $417.88. Esto me dice que tenemos muchas ventas básicas o de bajo valor. Las ventas grandes son la excepción, no la regla.

    Marketing No Funciona: El hallazgo más crítico es que el dinero que gastamos en marketing tiene una correlación de -0.00 con el ingreso. En términos simples, el costo de la publicidad no tiene ningún impacto medible en lo que vendemos. Necesitamos detener ese desperdicio inmediatamente.

    Tenemos Temporada Alta: Las ventas alcanzan sus picos máximos en Mayo y Marzo. Esto nos da una ventaja para planificar.

Mi Plan de Acción Inmediato

Mis recomendaciones se enfocan en solucionar el problema de la venta pequeña y el desperdicio en marketing.
A. Aumentar el Valor de Cada Pedido (Foco Principal)

Dado que aumentar el volumen es lo que más rápido nos genera dinero, propongo dos acciones para que la gente compre más:

    Venta Cruzada (Cross-Selling): Esto es como cuando te ofrezco "las papas con tu hamburguesa". Significa usar el sistema para sugerir productos complementarios a la gente. Por ejemplo, si alguien compra algo de Electrodomésticos (nuestro motor de ventas), yo debo sugerirle algo de Electrónica (accesorios) o Decoración (para ambientar). No hay que dejar que la gente se vaya solo con una cosa.

    Venta Superior (Up-Selling): Esto es ofrecerles algo mejor. Si el cliente ve la versión básica de un producto, yo le muestro la versión "Premium" inmediatamente, explicando: "Por solo un poco más, te llevas esto que dura más o tiene mejores funciones". El objetivo es elevar esos $417.88 que gasta la mayoría de nuestros clientes.

B. Solucionar el Problema del Marketing

Mi recomendación es clara: detengamos el gasto ineficiente.

    Acción Inmediata: Voy a utilizar el reporte integrado para ver el rendimiento (el ROI) de cada canal de marketing que usamos (redes sociales, email, etc.).

    Decisión: Una vez que identifique los canales que no dan resultados (los que tienen $r \approx 0$), voy a proponer cortar ese presupuesto y reasignarlo por completo a los canales que sí nos traen clientes rentables.

C. Preparación para la Temporada Alta

Recomendación: Planificar la operación para no perder ventas en los meses clave.

    Acción: Debemos asegurar el inventario de nuestros productos líderes (Electrodomésticos y Electrónica) con anticipación, comenzando los pedidos y la logística a finales de Enero y principios de Febrero. Así, cuando lleguen Marzo y Mayo, que son los meses de mayor demanda, estaremos listos para vender sin quedarnos sin stock.